In [1]:
# import the libraries 
from __future__ import print_function
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import pandas as pd

# get the gpu to do the train
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# to supress warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] ='2'

Using TensorFlow backend.


In [2]:
# check for the gpu
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10350908620227136755
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3157314764
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18025754436791925372
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# number of classes
num_classes = 8

# number of batch size and the epochs
batch_size = 32
epochs = 15

# input image dimnsions
img_rows, img_cols = 250, 140
target_size = (img_rows, img_cols)
color_mode = 'rgb'

# checking for shape includding the channels
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 3)
    
# setting the file to be read
train_path = './train'
valid_path = './valid'
test_path_1 = './test1'
test_path_2 = './test2'

In [4]:
# image Augmentation
datagen = ImageDataGenerator(rescale=1/255, shear_range=0.2, horizontal_flip=True, vertical_flip=True)

# loading the training data
training_data = datagen.flow_from_directory(train_path, target_size = target_size, batch_size = batch_size,
                                            class_mode = 'categorical', color_mode = color_mode, shuffle = True)

Found 3301 images belonging to 8 classes.


In [5]:
# image Augmentation
datagen = ImageDataGenerator(rescale=1/255)

# loading the validation data
validation_data = datagen.flow_from_directory(valid_path, target_size = target_size, batch_size = batch_size,
                                            class_mode = 'categorical', color_mode = color_mode, shuffle = True)

Found 476 images belonging to 8 classes.


In [6]:
# image Augmentation
datagen = ImageDataGenerator(rescale=1/255)

# loading the test data
test_data_1 = datagen.flow_from_directory(test_path_1, target_size = target_size, batch_size = 50,
                                            class_mode = None, color_mode = color_mode, shuffle = False)

Found 1000 images belonging to 1 classes.


In [7]:
# image Augmentation
datagen = ImageDataGenerator(rescale=1/255)

# loading the test data
test_data_2 = datagen.flow_from_directory(test_path_2, target_size = target_size, batch_size = 3,
                                            class_mode = None, color_mode = color_mode, shuffle = False)

Found 12153 images belonging to 1 classes.


In [8]:
# define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 138, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 124, 69, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 67, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 33, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128832)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16490624  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [10]:
# compile the loss funcation and the optimizer funcation
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [11]:
training_data.batch_size

32

In [17]:
# train the model
STEP_SIZE_TRAIN = training_data.n // training_data.batch_size
STEP_SIZE_VALID = validation_data.n // validation_data.batch_size
model.fit_generator(generator = training_data,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data = validation_data,
                    validation_steps = STEP_SIZE_VALID,
                    epochs = epochs
)

Epoch 1/15
103/103 [==============================] - 161s 2s/step - loss: 1.8643 - acc: 0.4505 - val_loss: 1.3972 - val_acc: 0.5580
Epoch 2/15
103/103 [==============================] - 160s 2s/step - loss: 1.2155 - acc: 0.5778 - val_loss: 0.9628 - val_acc: 0.6667
Epoch 3/15
103/103 [==============================] - 156s 2s/step - loss: 0.9021 - acc: 0.6880 - val_loss: 0.8140 - val_acc: 0.7117
Epoch 4/15
103/103 [==============================] - 140s 1s/step - loss: 0.7158 - acc: 0.7658 - val_loss: 0.6689 - val_acc: 0.7613
Epoch 5/15
103/103 [==============================] - 140s 1s/step - loss: 0.5933 - acc: 0.7996 - val_loss: 0.5425 - val_acc: 0.8221
Epoch 6/15
103/103 [==============================] - 140s 1s/step - loss: 0.5034 - acc: 0.8371 - val_loss: 0.5448 - val_acc: 0.8378
Epoch 7/15
103/103 [==============================] - 139s 1s/step - loss: 0.4605 - acc: 0.8516 - val_loss: 0.4605 - val_acc: 0.8671
Epoch 8/15
103/103 [==============================] - 140s 1s/step - 

In [12]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [13]:
sample_data_csv = pd.read_csv('sample_submission_stg1.csv')
sample_data_csv.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
1,img_00007.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283


In [14]:
sample_data_csv.shape

(1000, 9)

In [15]:
training_data.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

In [23]:
new_model_1 = load_model('my_model.h5')

In [24]:
new_model_2 = load_model('my_2nd_model.h5')

In [25]:
new_model_3 = load_model('my_3rd_model.h5')

In [33]:
img = image.load_img('shark.JPG', target_size=target_size, color_mode='rgb')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
new_model_1.predict(x)

array([[4.2243691e-12, 1.0000000e+00, 2.6926909e-12, 3.7977857e-32,
        1.5483947e-13, 3.6005875e-12, 4.7281506e-10, 2.5129066e-30]],
      dtype=float32)

In [19]:
test_data_1.reset()
predictions_1 = model.predict_generator(test_data_1, steps=20, verbose=0)

In [20]:
predictions_1[:2]

array([[0.11841138, 0.12217198, 0.10922462, 0.13547026, 0.1312581 ,
        0.1390277 , 0.1159405 , 0.12849544],
       [0.11432936, 0.11355143, 0.11437988, 0.13000454, 0.14077325,
        0.13430344, 0.11239614, 0.14026193]], dtype=float32)

In [21]:
np.savetxt('predictions_1.txt', predictions_1)

In [22]:
FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

test_image_list = test_data_1.filenames
f_submit = open('submit_1.csv', 'w')
f_submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')

for i, image_name in enumerate(test_image_list):
    pred = ['%.6f' % p for p in predictions_1[i, :]]
    f_submit.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))

f_submit.close()

In [23]:
df_1 = pd.read_csv('submit_1.csv')
df_1.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.118411,0.122172,0.109225,0.135470,0.131258,0.139028,0.115941,0.128495
1,img_00007.jpg,0.114329,0.113551,0.114380,0.130005,0.140773,0.134303,0.112396,0.140262


In [25]:
test_data_2.reset()
predictions_2 = model.predict_generator(test_data_2, steps=4051, verbose=0)
predictions_2[:2]

array([[0.11149997, 0.12139884, 0.1088907 , 0.13855469, 0.13002291,
        0.14042646, 0.11243585, 0.13677053],
       [0.10830174, 0.11919975, 0.1085957 , 0.13161685, 0.1364923 ,
        0.15086187, 0.11181303, 0.13311882]], dtype=float32)

In [26]:
FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

test_image_list = test_data_2.filenames
f_submit = open('submit_2.csv', 'w')
f_submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')

for i, image_name in enumerate(test_image_list):
    pred = ['%.6f' % p for p in predictions_2[i, :]]
    f_submit.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))

f_submit.close()

In [27]:
df_2 = pd.read_csv('submit_2.csv')
df_2.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,image_00001.jpg,0.111500,0.121399,0.108891,0.138555,0.130023,0.140426,0.112436,0.136771
1,image_00002.jpg,0.108302,0.119200,0.108596,0.131617,0.136492,0.150862,0.111813,0.133119


In [29]:
df_2['image'] = df_2['image'].map(lambda x: 'test_stg2/' + x)

In [30]:
df_2.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.111500,0.121399,0.108891,0.138555,0.130023,0.140426,0.112436,0.136771
1,test_stg2/image_00002.jpg,0.108302,0.119200,0.108596,0.131617,0.136492,0.150862,0.111813,0.133119


In [37]:
np.savetxt('predictions_2.txt', predictions_2)

In [31]:
new_df = pd.concat([df_1, df_2])

In [32]:
new_df.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.118411,0.122172,0.109225,0.135470,0.131258,0.139028,0.115941,0.128495
1,img_00007.jpg,0.114329,0.113551,0.114380,0.130005,0.140773,0.134303,0.112396,0.140262


In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13153 entries, 0 to 12152
Data columns (total 9 columns):
image    13153 non-null object
ALB      13153 non-null float64
BET      13153 non-null float64
DOL      13153 non-null float64
LAG      13153 non-null float64
NoF      13153 non-null float64
OTHER    13153 non-null float64
SHARK    13153 non-null float64
YFT      13153 non-null float64
dtypes: float64(8), object(1)
memory usage: 1.0+ MB


In [34]:
new_df.reset_index(drop=True, inplace=True)

In [35]:
new_df.head(2)

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.118411,0.122172,0.109225,0.135470,0.131258,0.139028,0.115941,0.128495
1,img_00007.jpg,0.114329,0.113551,0.114380,0.130005,0.140773,0.134303,0.112396,0.140262


In [36]:
new_df.to_csv('f.csv')